### Load libraries

In [ ]:

import sys
sys.path.insert(0, '../scripts/')
from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model
from load_modify_sample_utils import sample_dingo, sample_optgp
from correlations_utils import correlated_reactions, plot_correlation_matrix, split_forward_reverse
from correlations_utils import find_reactants_products, sharing_metabolites_square_matrix


/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/touliopoulos/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/home/touliopoulos/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Load model

In [2]:
ec_cobra_model, ec_cobra_reactions, ec_dingo_model, ec_dingo_reactions = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)

Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

### Modify model and sample

In [3]:
ec_cobra_model_condition_1, ec_dingo_model_condition_1 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=100)
ec_cobra_model_condition_2, ec_dingo_model_condition_2 = modify_model(ec_cobra_model, objective_function="BIOMASS_Ecoli_core_w_GAM", optimal_percentage=0)


samples_dingo_condition_1 = sample_dingo(ec_dingo_model_condition_1, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_1.shape)

samples_dingo_condition_2 = sample_dingo(ec_dingo_model_condition_2, reaction_in_rows = True, ess=2000)
print(samples_dingo_condition_2.shape)



Read LP format model from file /tmp/tmpziepbda3.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


Read LP format model from file /tmp/tmps7ryeu_w.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
phase 1: number of correlated samples = 500, effective sample size = 29, ratio of the maximum singilar value over the minimum singular value = 893.519
phase 2: number of correlated samples = 500, effective sample size = 4, ratio of the maximum singilar value over the minimum singular value = 454.051
phase 3: number of correlated samples = 500, effective sample size = 7, ratio of the maximum singilar value over the minimum singular value = 205.366
phase 4: number of correlated samples = 500, effective sample size = 172, ratio of the maximum singilar value over the minimum singular value = 23.8708
phase 5: number of correlated samples = 500, effective sample size = 141, ratio of the maximum singilar value over the minimum singular value = 2.979

[5]maximum marginal PSRF: 1.12073



[5]total ess 2079: number of correlated samples = 6600


(95, 6600)
phase 1: number of correlated samples = 500, effective sample size = 25, ratio of the maximum singilar value over the minimum singular value = 1131.05
phase 2: number of correlated samples = 500, effective sample size = 133, ratio of the maximum singilar value over the minimum singular value = 2.02466
phase 3: number of correlated samples = 2400, effective sample size = 896
phase 4: number of correlated samples = 2200, effective sample size = 962
[5]total ess 2016: number of correlated samples = 5600


(95, 5600)


[5]maximum marginal PSRF: 1.0038


### Split reactions having both positive and negative flux values (reversible reactions) into two separate reactions (forward and reverse).

In [4]:

extended_steady_states, extended_reactions = split_forward_reverse(samples_dingo_condition_2, reactions=ec_dingo_reactions)


reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions = find_reactants_products(ec_cobra_model, extended_reactions)
print(len(reversibility_list_all_reactions), len(reactants_list_all_reactions))



127 127


### Calculate a boolean matrix with dimensions equal to the number of reactions (here, reactions after the forward and reverse split). "True" in a position of the 2D array represents 2 reactions with a sharing reactant or product. This is done to filter correlations between reactions that do not share metabolites

In [5]:

boolean_sharing_metabolites_matrix = sharing_metabolites_square_matrix(extended_reactions, reversibility_list_all_reactions, reactants_list_all_reactions, products_list_all_reactions)


print(boolean_sharing_metabolites_matrix)
print(boolean_sharing_metabolites_matrix.shape)


[[ True False  True ... False False False]
 [False  True False ... False False False]
 [ True False  True ... False False False]
 ...
 [False False False ...  True False False]
 [False False False ... False  True False]
 [False False False ... False False  True]]
(127, 127)


### Compute pairwise linear and non-linear correlations from flux samples and filter (remove) correlations between reactions not sharing any metabolites

In [11]:


(linear_correlation_matrix, 
non_linear_correlation_matrix, 
mixed_correlation_matrix, 
correlations_dictionary) = correlated_reactions(
        steady_states = extended_steady_states,
        boolean_sharing_metabolites_matrix=boolean_sharing_metabolites_matrix,
        reactions=extended_reactions, 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True
)




/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning:

invalid value encountered in divide

/home/touliopoulos/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning:

invalid value encountered in divide



Completed the process of 1 from 7003 copulas
Completed the process of 2 from 7003 copulas
Completed the process of 3 from 7003 copulas
Completed the process of 4 from 7003 copulas
Completed the process of 5 from 7003 copulas
Completed the process of 6 from 7003 copulas
Completed the process of 7 from 7003 copulas
Completed the process of 8 from 7003 copulas
Completed the process of 9 from 7003 copulas
Completed the process of 10 from 7003 copulas
Completed the process of 11 from 7003 copulas
Completed the process of 12 from 7003 copulas
Completed the process of 13 from 7003 copulas
Completed the process of 14 from 7003 copulas
Completed the process of 15 from 7003 copulas
Completed the process of 16 from 7003 copulas
Completed the process of 17 from 7003 copulas
Completed the process of 18 from 7003 copulas
Completed the process of 19 from 7003 copulas
Completed the process of 20 from 7003 copulas
Completed the process of 21 from 7003 copulas
Completed the process of 22 from 7003 copul

### Show some pairs of reactions with extreme non-linear dependencies

In [9]:

filtered_positive = {
    pair: metrics
    for pair, metrics in correlations_dictionary.items()
    if metrics['jensenshannon'] > 0.10 and metrics['indicator'] > 1.2
}

print(filtered_positive)
print(filtered_positive.keys())


filtered_negative = {
    pair: metrics
    for pair, metrics in correlations_dictionary.items()
    if metrics['jensenshannon'] < -0.10 and metrics['indicator'] < 1.2

}

print(filtered_negative)
print(filtered_negative.keys())


{'PGI~PFK': {'pearson': 0, 'jensenshannon': 0.10914854649144647, 'indicator': 1.5663401574608151, 'classification': 'positive_upper_tail'}, 'PGK_rev~PFK': {'pearson': 0, 'jensenshannon': 0.11404012424015016, 'indicator': 1.6943396189733007, 'classification': 'positive_upper_tail'}, 'PGM_rev~PFK': {'pearson': 0, 'jensenshannon': 0.11309012416596398, 'indicator': 1.6936090189058173, 'classification': 'positive_upper_lower_tail'}, 'RPE_rev~PFK': {'pearson': 0, 'jensenshannon': 0.1093395180841819, 'indicator': 1.5996376781177801, 'classification': 'positive_upper_tail'}, 'RPI~PFK': {'pearson': 0, 'jensenshannon': 0.10964616280406729, 'indicator': 1.5969341719180965, 'classification': 'positive_upper_tail'}, 'ENO~PFK': {'pearson': 0, 'jensenshannon': 0.11309012416596398, 'indicator': 1.6936090189058173, 'classification': 'positive_upper_lower_tail'}, 'TALA_rev~PFK': {'pearson': 0, 'jensenshannon': 0.10901042006568563, 'indicator': 1.6032608665051986, 'classification': 'positive_upper_tail'}

### Plot 3 correlation matrices, (a) only with linear correlations, (b) only with non-linear correlations and (c) with both linear and non-linear correlations

In [10]:

cobra_reactions_str = [str(reaction.id) for reaction in ec_cobra_model.reactions]

plot_correlation_matrix(linear_correlation_matrix, extended_reactions)
plot_correlation_matrix(non_linear_correlation_matrix, extended_reactions)
plot_correlation_matrix(mixed_correlation_matrix, extended_reactions)


### Calculate pairwise correlatins without filtering for sharing metabolites

In [12]:

(linear_correlation_matrix_no_metabolites, 
non_linear_correlation_matrix_no_metabolites, 
mixed_correlation_matrix_no_metabolites, 
correlations_dictionary_no_metabolites) = correlated_reactions(
        steady_states = extended_steady_states,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions, 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)


Completed the process of 1 from 7003 copulas
Completed the process of 2 from 7003 copulas
Completed the process of 3 from 7003 copulas
Completed the process of 4 from 7003 copulas
Completed the process of 5 from 7003 copulas
Completed the process of 6 from 7003 copulas
Completed the process of 7 from 7003 copulas
Completed the process of 8 from 7003 copulas
Completed the process of 9 from 7003 copulas
Completed the process of 10 from 7003 copulas
Completed the process of 11 from 7003 copulas
Completed the process of 12 from 7003 copulas
Completed the process of 13 from 7003 copulas
Completed the process of 14 from 7003 copulas
Completed the process of 15 from 7003 copulas
Completed the process of 16 from 7003 copulas
Completed the process of 17 from 7003 copulas
Completed the process of 18 from 7003 copulas
Completed the process of 19 from 7003 copulas
Completed the process of 20 from 7003 copulas
Completed the process of 21 from 7003 copulas
Completed the process of 22 from 7003 copul

In [13]:

filtered_g = {
    pair: metrics
    for pair, metrics in correlations_dictionary_no_metabolites.items()
    if metrics['jensenshannon'] > 0.1 and metrics['indicator'] > 1.2
}

print(filtered_g)
print(filtered_g.keys())


filtered_l = {
    pair: metrics
    for pair, metrics in correlations_dictionary_no_metabolites.items()
    if metrics['jensenshannon'] < -0.1 and metrics['indicator'] < 1.2

}

print(filtered_l)
print(filtered_l.keys())


{'PGI~PFK': {'pearson': 0, 'jensenshannon': 0.14130783223355656, 'indicator': 1.6395262741810903, 'classification': 'positive_upper_lower_tail'}, 'PGK_rev~PFK': {'pearson': 0, 'jensenshannon': 0.1395066532318288, 'indicator': 1.651403246936589, 'classification': 'positive_upper_lower_tail'}, 'PGM_rev~PFK': {'pearson': 0, 'jensenshannon': 0.13836270950948815, 'indicator': 1.6548410911551292, 'classification': 'positive_upper_lower_tail'}, 'ACONTa~ACALD': {'pearson': 0, 'jensenshannon': 0.10794260293247158, 'indicator': 1.74270352983169, 'classification': 'positive_upper_lower_tail'}, 'ACONTb~ACALD': {'pearson': 0, 'jensenshannon': 0.10794260293247158, 'indicator': 1.74270352983169, 'classification': 'positive_upper_lower_tail'}, 'RPE_rev~PFK': {'pearson': 0, 'jensenshannon': 0.13648306143974281, 'indicator': 1.6046681229860382, 'classification': 'positive_upper_lower_tail'}, 'CS~ACALD': {'pearson': 0, 'jensenshannon': 0.10794260293247158, 'indicator': 1.74270352983169, 'classification':

In [14]:

cobra_reactions_str = [str(reaction.id) for reaction in ec_cobra_model.reactions]

plot_correlation_matrix(linear_correlation_matrix_no_metabolites, extended_reactions)
plot_correlation_matrix(non_linear_correlation_matrix_no_metabolites, extended_reactions)
plot_correlation_matrix(mixed_correlation_matrix_no_metabolites, extended_reactions)
